In [ ]:
#-------Set up environment--------
options(warn = -1, verbose=FALSE)
#Load packages
library(Seurat)
library(dplyr)
library(SeuratWrappers)
library(harmony)
library(MAST)
library(cowplot)
library(stringi) #string manipulation such as stri_sub
library(dplyr) #data manipulation library
library(ggplot2)
library(readr)
ulimit::memory_limit(100000)
setwd("/data/2320454f/jupyter-2320454f/BIORRA")
set.tempdir("/data/2320454f/td")

In [ ]:
#Read in experiment metadata
metadata <- read_csv("Rhapsody-MetadataforBIORRAstudy_210927.csv")
metadata <- metadata[1:24,]

In [ ]:
#Create a list of the files from target directory
run1 <- "/data/2320454f/jupyter-2320454f/BIORRA/"
fileList <- list.files(path=run1, recursive=TRUE)

#Keep only files of interest (i.e. those which contain UMInormalized counts and are sample tagged)
foi <- fileList[grepl("RSEC_MolsPerCell.csv", fileList)]     
sampleTagged<- foi[grepl("SampleTag", foi)] 

#Iterate through files to read in samples
lib <- list()
names <- list()
newnames <- list()
experiment1 <- list()
for (i in 1:length(sampleTagged)) {
  #Get the sample name only 
  names[[i]] <- gsub("_RSEC_MolsPerCell[.]csv","", sampleTagged[[i]])
  newnames[[i]] <- gsub(".*hs_", "", names[[i]])
  
  #Read in each file under new sample name
  lib[[newnames[[i]]]] <- read_csv(paste0(run1,sampleTagged[[i]]), skip = 7)
  
  #Rename feature names
  colnames(lib[[i]]) <- gsub("[|].*","", colnames(lib[[i]]))
  
  countsData <- as.data.frame(lib[[i]])
  rownames(countsData) <- countsData$Cell_Index
  countsData <- countsData[,-1]
  
  #Create a seurat object for each sample tag
  experiment1[[i]] <- CreateSeuratObject(counts = t(countsData)) 
  
  #Add some metadata
  experiment1[[i]]$sample <- names(lib[i])
  meta <- metadata[which(metadata$`SampleID`%in%names(lib[i])),]
  experiment1[[i]]$FinalOutcome <- meta$FinalOutcome
  experiment1[[i]]$Experiment <- meta$Experiment
  experiment1[[i]]$Tag <- meta$Tag    
  experiment1[[i]]$Condition <- meta$Condition 
  experiment1[[i]]$PatientID <- meta$PatientID
  experiment1[[i]]$Date <- meta$Date
  experiment1[[i]]$Timepoint <- meta$Timepoint
  experiment1[[i]]$SampleCollection <- meta$SampleCollection
}

In [ ]:
#Merge Objects and carry out standard processing and visualisation
experiment1.merged <- merge(experiment1[[1]], y = c(experiment1[2:length(sampleTagged)]), add.cell.ids=(names(names)), project="BIORRA EXPERIMENT 1")
experiment1.merged <- NormalizeData(experiment1.merged)
experiment1.merged <- FindVariableFeatures(experiment1.merged)
experiment1.merged <- ScaleData(object = experiment1.merged, verbose  = FALSE)
experiment1.merged <- RunPCA(object = experiment1.merged, npcs = 30, verbose = FALSE)

In [ ]:
options(repr.plot.width=10, repr.plot.height=20)
#Visualise PCs
PCHeatmap(experiment1.merged,dims=16:30,cells=500)

In [ ]:
#Run Standard Clustering Pipeline
experiment1.merged <- RunUMAP(object = experiment1.merged, reduction = "pca", dims = 1:30)
experiment1.merged <- FindNeighbors(object = experiment1.merged, reduction = "pca", dims = 1:30)
experiment1.merged <- FindClusters(experiment1.merged, resolution = 0.8)

In [ ]:
#Run Harmony And Integrate By Sample
biorra.harmony<-RunHarmony(experiment1.merged, group.by.vars="sample")

In [ ]:
##UMAP Resolution 0.6
biorra.harmony <- RunUMAP(object = biorra.harmony, reduction = "harmony", dims = 1:30)
biorra.harmony <- FindNeighbors(object = biorra.harmony, reduction = "harmony", dims = 1:30)
biorra.harmony <- FindClusters(biorra.harmony, resolution = 1)

In [ ]:
#Visualise UMAP by Clusters and Sample
options(repr.plot.width=10, repr.plot.height=8)
DimPlot(object = biorra.harmony, reduction = "umap", label = TRUE) + theme_linedraw() + theme(strip.text = element_text(size = 15, face="bold"), strip.background = element_rect(), panel.grid.major = element_blank(), panel.grid.minor = element_blank(),panel.background = element_blank())
options(repr.plot.width=10, repr.plot.height=8)
DimPlot(object = biorra.harmony, reduction = "umap", label = TRUE, group.by = "PatientID") + theme_linedraw() + theme(strip.text = element_text(size = 15, face="bold"), strip.background = element_rect(), panel.grid.major = element_blank(), panel.grid.minor = element_blank(),panel.background = element_blank())

#Split by Outcome Regardless of Timepoint
options(repr.plot.width=10, repr.plot.height=8)
DimPlot(object = biorra.harmony, reduction = "umap", label = TRUE, split.by = "Condition") + theme_linedraw() + theme(strip.text = element_text(size = 15, face="bold"), strip.background = element_rect(), panel.grid.major = element_blank(), panel.grid.minor = element_blank(),panel.background = element_blank())

In [ ]:
#Visualise QC Metrics
VlnPlot(biorra.harmony, features=c("nCount_RNA"), group.by="PatientID", pt.size=0) + stat_summary(fun.y = median, geom='point', size = 2, colour = "black")
VlnPlot(biorra.harmony, features=c("nFeature_RNA"), group.by="PatientID", pt.size=0) + stat_summary(fun.y = median, geom='point', size = 2, colour = "black")

In [ ]:
# find markers for every cluster compared to all remaining cells, report only the positive ones
biorra.harmony.markers <- FindAllMarkers(object = biorra.harmony, only.pos = TRUE, min.pct = 0.4, logfc.threshold = 0.25)
write.csv("biorra.harmony.markers-MAST-min.pct0.4-res0.2.csv")
top10 <- biorra.harmony.markers %>% group_by(cluster) %>% top_n(n = 10, wt = avg_logFC)
options(repr.plot.width=20, repr.plot.height=15)
DoHeatmap(object = biorra.harmony, features = top10$gene)

In [ ]:
options(repr.plot.width=20, repr.plot.height=15)
#Visualise some key markers of interest
VlnPlot(object = biorra.harmony, features = c("CD1C", "CD163", "CD36", "CD14", "CLEC10A", "CD33", "CADM1"),pt.size=0)

In [ ]:
#Rename Seurat Clusters to Celltypes based on Marker Genes
biorra.harmony$celltype <- plyr::revalue(as.character(biorra.harmony$seurat_clusters),
                                    c("0"='CD163 CD69 CD1c Low Dendritic Cells',
                                      "1"='CD16 Monocytes',
                                      "2"='CLEC4E CD163 IL-1B CD14 Monocytes',
                                      "3"='CXCL8 CD14 Monocytes',
                                      "4"='CD1c High Dendritic Cells',
                                      "5"='CCL3 CD14 Monocytes',
                                      "6"='S100A12 CD14 Monocytes',
                                      "7"='CD14 CD16 Monocytes',
                                      "8"='pDC',
                                      "9"='C1QA C1QB CD16 Monocytes',
                                      "10"='CLEC10A CD33 Dendritic Cells',
                                      "11"='IRF8 DPP4 Cells',
                                      "12"='NK Cells',
                                      "13"='AXL CD5 CD1c Dendritic Cells', 
                                      "14"='B Cells',
                                      "15"='Plasma Cells'))
Idents(biorra.harmony) <- "celltype"

#Visualise UMAP with new annotations
options(repr.plot.width=10, repr.plot.height=8)
DimPlot(object = biorra.harmony, reduction = "umap", label = FALSE) + theme_linedraw() + theme(strip.text = element_text(size = 15, face="bold"), strip.background = element_rect(), panel.grid.major = element_blank(), panel.grid.minor = element_blank(),panel.background = element_blank())

In [ ]:
#Save Integrated Object with All Cells
saveRDS(biorra.harmony, "biorra.harmony.rds")

In [ ]:
#Remove all non-myeloid cells
myeloid.biorrah<-subset(biorra.harmony, idents=c("NK Cells", "B Cells", "Plasma Cells", "pDC", "IRF8 DPP4 Cells"), invert=TRUE)

In [ ]:
#Save Only the Myeloid Cells from BioRRA
saveRDS(myeloid.biorrah, "myeloid.biorrah.rds")

# Now Begins re-analysis of Myeloid Cells Only

In [ ]:
#-------Set up environment--------
options(warn = -1, verbose=FALSE)
#Load packages
library(Seurat)
library(dplyr)
library(MAST)
library(ggrepel)
library(cowplot)
library(stringi) #string manipulation such as stri_sub
library(dplyr) #data manipulation library
library(ggplot2)
library(ggmin)
library(readr)
ulimit::memory_limit(100000)
setwd("/data/2320454f/jupyter-2320454f/BIORRA")
set.tempdir("/data/2320454f/td")

In [ ]:
myeloid.biorrah <- readRDS("myeloid.biorrah.rds")
myeloid.biorrah

In [ ]:
myeloid.biorrah <- FindVariableFeatures(myeloid.biorrah)
myeloid.biorrah <- ScaleData(object = myeloid.biorrah, verbose  = FALSE)
myeloid.biorrah <- RunPCA(object = myeloid.biorrah, npcs = 30, verbose = FALSE) 

In [ ]:
#Visualise dimensionality of the data with PCA Plot
options(repr.plot.width=10, repr.plot.height=20)
PCHeatmap(myeloid.biorrah,dims=1:15,cells=500)

In [ ]:
#Run the standard workflow for visualization and clustering
myeloid.biorrah <- RunUMAP(object = myeloid.biorrah, reduction = "harmony", dims = 1:20)
myeloid.biorrah <- FindNeighbors(object = myeloid.biorrah, reduction = "harmony", dims = 1:20)
myeloid.biorrah <- FindClusters(myeloid.biorrah, resolution = 0.9)

In [ ]:
#Make New Column in Metadata
myeloid.biorrah$condition_timepoint <- paste0(myeloid.biorrah$Condition, "_", myeloid.biorrah$Timepoint)
#Make New Column in Metadata
myeloid.biorrah$condition_state <- paste0(myeloid.biorrah$condition_timepoint, "_", myeloid.biorrah$FinalOutcome)
#Rename
myeloid.biorrah$remission.state <- plyr::revalue(as.character(myeloid.biorrah$condition_state),
c('Remission_1st_Flare'="Baseline Remission (Flare Endpoint)",'Remission_1st_Maintained Remission'="Baseline Remission (Sustained Remission Endpoint)",'Flare_2nd_Flare'="Flare", 'Remission_2nd_Maintained Remission'="Sustained Remission"))

In [ ]:
#UMAP by Cluster
options(repr.plot.width=10, repr.plot.height=8)
DimPlot(object = myeloid.biorrah, reduction = "umap", label = TRUE) + theme_linedraw() + theme(strip.text = element_text(size = 15, face="bold"), strip.background = element_rect(), panel.grid.major = element_blank(), panel.grid.minor = element_blank(),panel.background = element_blank())

#UMAP Split by Disease Outcome
options(repr.plot.width=30, repr.plot.height=20)
DimPlot(object = myeloid.biorrah, reduction = "umap", label = TRUE, split.by = "remission.state", ncol=4, pt.size=1) + theme_linedraw() + theme(strip.text = element_text(size = 15, face="bold"), strip.background = element_rect(), panel.grid.major = element_blank(), panel.grid.minor = element_blank(),panel.background = element_blank())

In [ ]:
#Create Feature Plot with GOI to give initial idea of Clusters
DefaultAssay(myeloid.biorrah) <- "RNA"
options(repr.plot.width=15, repr.plot.height=8)
FeaturePlot(myeloid.biorrah, features=c("CD163","CD1C","CD14","CLEC9A", "CLEC10A"), order=TRUE)

In [ ]:
DefaultAssay(myeloid.biorrah) <- "RNA"
myeloid.biorrah.markers <- FindAllMarkers(myeloid.biorrah, only.pos=TRUE, min.pct=0.4, test.use="MAST")

#How many clusters?
num.clust <- length(unique(myeloid.biorrah$seurat_clusters))
num.clust

myeloid.biorrah.markers$multiTestCorrected <- myeloid.biorrah.markers$p_val_adj*num.clust
myeloid.biorrah.markers <- myeloid.biorrah.markers[which(myeloid.biorrah.markers$multiTestCorrected<0.05),]
top15 <- myeloid.biorrah.markers %>% group_by(cluster) %>% top_n(n = 15, wt = avg_logFC)
write.csv(myeloid.biorrah.markers, "myeloid.biorrah.markers-MAST-15dims-res1.0-minpct0.4.csv")

In [ ]:
#Remember to change to integrated assay to do heatmap 
DefaultAssay(myeloid.biorrah) <- 'RNA'
options(repr.plot.width=20, repr.plot.height=30)
DoHeatmap(myeloid.biorrah, features=top15$gene) + 
    theme(text = element_text(size = 20))

In [ ]:
BuildClusterTree(myeloid.biorrah)%>%PlotClusterTree()

In [ ]:
#Violin Plot for Useful Markers
options(repr.plot.width=17, repr.plot.height=10)
VlnPlot(myeloid.biorrah, features=c("CD163", "CD36", "CD1C", "FCGR3A", "IRF8", "IRF4", "CD1C", "IL3RA", "CD33"), pt.size=0) + stat_summary(fun.y = median, geom='point', size = 2, colour = "black")

In [ ]:
#Rename Clusters and put in desired order
myeloid.biorrah$celltype <- plyr::revalue(as.character(myeloid.biorrah$seurat_clusters),
                                    c("0"='CD16 Monocytes',
                                      "1"='CXCL8 CD14 Monocytes',
                                      "2"='CLEC4E CD163 IL1B CD14 Monocytes',
                                      "3"='CD1c High Dendritic Cells',
                                      "4"='S100A9hi S100A12hi CD14 Monocytes',
                                      "5"='CD163 CD1c Low Dendritic Cells',
                                      "6"='IRF4hi CD163 CD1c Dendritic Cells',
                                      "7"='CD14 CD16 Monocytes',
                                      "8"='CCL3 CD14 Monocytes',
                                      "9"='C1QA C1QB CD16 Monocytes',
                                      "10"='CLEC10A CD33 CD1c Dendritic Cells',
                                      "11"='AXL CD5 IL18R1 CD1c Dendritic Cells'))

In [ ]:
#Move to new object
mb<-myeloid.biorrah

#Merge Monocyte Clusters as they are transcriptionally similar in gene panel
mb$celltype <- plyr::revalue(as.character(mb$seurat_clusters),
                                    c("CD16 Monocytes"='CD16 Monocytes',
                                      "CXCL8 CD14 Monocytes"='CXCL8 CCL3 Activated CD14 Monocytes',
                                      "CLEC4E CD163 IL1B CD14 Monocytes"='CXCL8 CCL3 Activated CD14 Monocytes',
                                      "CD1c High Dendritic Cells"='CD1c High Dendritic Cells',
                                      "S100A9hi S100A12hi CD14 Monocytes"='S100A9hi S100A12hi Classical CD14 Monocytes',
                                      "CD163 CD1c Low Dendritic Cells"='CD163 CD1c Low Dendritic Cells',
                                      "IRF4hi CD163 CD1c Dendritic Cells"='IRF4hi CD163 CD1c Dendritic Cells',
                                      "CD14 CD16 Monocytes"='CD14 CD16 Monocytes',
                                      "CCL3 CD14 Monocytes"='CXCL8 CCL3 Activated CD14 Monocytes',
                                      "C1QA C1QB CD16 Monocytes"='C1QA C1QB CD16 Monocytes',
                                      "CLEC10A CD33 CD1c Dendritic Cells"='CLEC10A CD33 CD1c Dendritic Cells',
                                      "AXL CD5 IL18R1 CD1c Dendritic Cells"='AXL CD5 IL18R1 CD1c Dendritic Cells'

#Rename Clusters and put in desired order
levels(x = mb) <- c("CXCL8 CCL3 Activated CD14 Monocytes", "S100A9hi S100A12hi Classical CD14 Monocytes",
                                                                                 "CD14 CD16 Monocytes",
                                                                                "CD16 Monocytes",
                                                                                "C1QA C1QB CD16 Monocytes",
                                                                                "CD1c High Dendritic Cells",
                                                                                "IRF4hi CD163 CD1c Dendritic Cells",
                                                                                "CD163 CD1c Low Dendritic Cells",
                                                                                "AXL CD5 IL18R1 CD1c Dendritic Cells",
                                                                                "CLEC10A CD33 CD1c Dendritic Cells")

In [ ]:
clus.cols<-c("red", "#00FFFF", "yellow", "#355E3B", "#7CFC00", "#9400D3", "#800000", "orange", "#66CDAA", "#191970")

In [ ]:
#Proportions of cells in each cluster per group
freq_table_group <- as.data.frame(prop.table(x = table(Idents(mb), mb@meta.data[, "remission.state"]), margin = 2))
options(repr.plot.width=10, repr.plot.height=10)
ggplot(data=freq_table_group, aes(x=freq_table_group$Var2, y=freq_table_group$Freq, fill=freq_table_group$Var1)) + theme_linedraw() + theme(legend.text=element_text(size=15))+
geom_bar(stat="identity", color="black") + labs(x="Group", y="Proportion of Cells", fill="Cluster") + theme(axis.text.x.bottom = element_text(angle=45,hjust=1,size=13), axis.ticks.x.bottom = element_blank(), panel.grid.major = element_blank(), panel.grid.minor = element_blank()) + theme(strip.text = element_text(size = 15, face="bold"), strip.background = element_rect())+ scale_fill_manual(values=clus.cols)

In [ ]:
BuildClusterTree(mb)%>%PlotClusterTree()

In [ ]:
#Find Markers for Merged Clusters
DefaultAssay(mb) <- "RNA"
mb.markers <- FindAllMarkers(mb, only.pos=TRUE, min.pct=0.4, test.use="MAST")

In [ ]:
#How many clusters?
num.clust <- length(unique(mb$celltype))
num.clust

mb.markers$multiTestCorrected <- mb.markers$p_val_adj*num.clust
mb.markers <- mb.markers[which(mb.markers$multiTestCorrected<0.05),]
top15 <- mb.markers %>% group_by(cluster) %>% top_n(n = 15, wt = avg_logFC)
write.csv(mb.markers, "mb.markers-MAST-15dims-res1.0-minpct0.4.csv")

In [ ]:
#Remember to change to integrated assay to do heatmap 
#when using seurat heatmap RNA better for identification
DefaultAssay(mb) <- "RNA"
options(repr.plot.width=23, repr.plot.height=23)
DoHeatmap(mb, features=top15$gene, label=FALSE, group.colors=clus.cols) + 
    theme(text = element_text(size = 25))

In [ ]:
#UMAP by Cluster
options(repr.plot.width=13, repr.plot.height=8)
DimPlot(object = mb, reduction = "umap", label = FALSE, cols = clus.cols,  label.size = 10) + theme_linedraw() + theme(strip.text = element_text(size = 15, face="bold"), strip.background = element_rect(), panel.grid.major = element_blank(), panel.grid.minor = element_blank(),panel.background = element_blank())

In [ ]:
#Make Data Frame to get Proportion Numbers for Prism Analysis
freq_table_sample_endpoint <- as.data.frame(prop.table(x = table(Idents(mb), mb@meta.data[, "sample"]), margin = 2))
freq_table_sample_endpoint
colnames(freq_table_sample_endpoint) <- c("Cluster","Sample","Freq")
#Save output of table and restructure for analysis
write.csv(reshape(freq_table_sample_endpoint, timevar="Sample",idvar="Cluster", direction="wide"),"cluster_prop.table_sample_endpoint.csv")

In [ ]:
#Prepare Heatmap Using Complex Heatmap Package
DefaultAssay(mb) <- "RNA"
mb.markers <- FindAllMarkers(mb, only.pos=TRUE, min.pct=0.4, test.use="MAST")

mb.markers <- mb.markers[which(mb.markers$p_val_adj<0.05),]

In [ ]:
library(data.table)
# pick signficant genes to show in the heatmap
# arrange the genes such that the gene with max logFC is at the center and values decrease as we move away from center
genes_use_df<-data.table(mb.markers)[
    p_val_adj<0.05][
    , .SD[order(-avg_log2FC)][1:20, ], by = cluster][
    , .SD[!is.na(gene)]][
    , .SD[order(avg_log2FC)], by = cluster][
    , ":=" (len = .N, rank = frank(avg_log2FC)), by = cluster][
    , .SD[c(seq(1, round(.SD$len[1]/3), 1), seq(.SD$len[1], round(.SD$len[1]/3)+1, -1)), ], by = cluster][
    , .(gene, avg_log2FC, cluster, p_val_adj) 
]

In [ ]:
#Organise genes we want to include by cluster
genes_use_df <- genes_use_df %>% distinct(gene, .keep_all = TRUE) %>% arrange(cluster)
genes_use_df[, .SD[order(-avg_log2FC)][1:5, ], by = cluster]

In [ ]:
message("Checking that the genes are unique: ", nrow(genes_use_df) == length(unique(genes_use_df$gene)))
message("Number of genes in heatmap: ", nrow(genes_use_df))

 

# select the top 5 genes with highest logFC in each cluster and assign each gene to a cell type based on it logFC - the gene belongs to the celltype with higher logFC
genes_to_mark_celltypes<-genes_use_df[, .SD[order(-avg_log2FC)][1:5, ], by = cluster][
    , .(cluster, gene)
]

 

.mat<-mb.markers %>% 
    subset(gene %in% genes_use_df$gene) %>% 
    dplyr::select(cluster, avg_log2FC, gene) %>% 
    tidyr::spread(cluster, avg_log2FC) %>% 
    tibble::column_to_rownames('gene') %>% 
    as.matrix() %>% t() 
.mat<-.mat[, genes_use_df$gene]

 

message(
    "Are all genes that are being marked also present in the matrix?: ", 
    length(setdiff(genes_to_mark_celltypes$gene, colnames(.mat))) == 0
)

 

genes_idx<-data.frame(idx = which(colnames(.mat) %in% genes_to_mark_celltypes$gene)) %>% 
    mutate(gene = colnames(.mat)[idx])

In [ ]:
#Get the exact position of genes we wish to annotate in the matrix
genes_to_mark_celltypes<-genes_to_mark_celltypes %>% left_join(unique(genes_idx), by = "gene")

genes_to_mark_celltypes %>% arrange(cluster)

In [ ]:
.mat[is.na(.mat)] <- 0

In [ ]:
names(clus.cols)<-levels(mb)
library(ComplexHeatmap)
options(repr.plot.width=15, repr.plot.height=5)

gene.anno<-columnAnnotation(
    col_ann = anno_mark(
        at = genes_to_mark_celltypes$idx,
        side = "bottom",
        labels = genes_to_mark_celltypes$gene)
    )

clust.anno<-columnAnnotation(show_legend = TRUE,
    cluster = genes_use_df$cluster,
    col = list(cluster=clus.cols))

In [ ]:
anno_mark(
        at = genes_to_mark_celltypes$idx,
        side = "bottom",
        labels = genes_to_mark_celltypes$gene)

In [ ]:
library(scales)

In [ ]:
split<-genes_use_df$cluster
options(repr.plot.width=40, repr.plot.height=5)
colnames(.mat)<-NULL
.mat %>% 
    Heatmap(
        name = 'avg_log2FC',
        col = circlize::colorRamp2(c(0, 0.1, 2), c('white', 'white', muted('blue'))),
        #column_names_gp = grid::gpar(fontsize = 10),
#         row_dend_side = "right",
        cluster_rows = FALSE,
        cluster_columns = FALSE,
        top_annotation = clust.anno,
        column_title=NULL,
        bottom_annotation = gene.anno,
        column_split = split,
    ) 